1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [21]:
# 
def convert(a):
    
    # binary representation
    a_bin = bin(a)
    print('Binary representation of', a, ':', a_bin)
    
    # decimal representation
    print('Decimal representation of', a_bin, ':', int(a_bin, 2))

    
    # hexadecimal representation
    a_hex = hex(a)
    print('Hexadecimal representation of', a, ':', a_hex)
    
    # decimal representation
    print('Decimal representation of', a_hex, ':', int(a_hex, 16))
    
    # binary representation
    a_bin = bin(int(a_hex, 16))
    print('Binary representation of', a_hex, ':', a_bin)
    
    # hexadecimal representation
    a_hex = hex(int(a_bin, 2))
    print('Hexadecimal representation of', a_bin, ':', a_hex)
    
    
        
        
a = 20
convert(a)

Binary representation of 20 : 0b10100
Decimal representation of 0b10100 : 20
Hexadecimal representation of 20 : 0x14
Decimal representation of 0x14 : 20
Binary representation of 0x14 : 0b10100
Hexadecimal representation of 0b10100 : 0x14


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [13]:
def convertion(word):
    sign = int(word[31])               #takes sign
    exp = word[23:31]
    exp = reversed(exp)                #exponent operations
    exp = "".join([str(i) for i in exp])
    exp_dec = int(exp,2)
    mant = word[0:23]                  #mantissa operations
    mant = list(mant)
    mant.reverse()
    mant_dec = 0
    
    for i in range(len(mant)):
        temp = int(mant[i])
        if temp == 1:
            mant_dec += pow(2,-(i+1))
            
    n = (1+mant_dec)*pow(2,exp_dec-127)
    
    res = 0
    if sign == 0:                 #result with addition of sign
        res = n
    else:
        res = -n    
    return res

word = "00000000000000000000010101010100"
print(convertion(word))

2.8421709430404007e-13


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [14]:
import math

double = 1.0
halve = 1.0
check_over = math.inf
check_under = 0.0
i = 0

while(double.hex() != check_over.hex()):
    double = double*2
    i += 1
print(i, "overflow iteration  =  ", "%2.5e"%double) #last number before overflow n = 1023

i = 0
while(halve.hex() != check_under.hex()):
    halve = halve/2
    i += 1
print(i, "underflow iteration  =  ", "%2.5e"%halve) #last number before underflow n = 1074

1024 overflow iteration  =   inf
1075 underflow iteration  =   0.00000e+00


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [15]:
n = 200
smaller = 1
original = 20.0
flag = True
i = 0 

while(i in range(n) and flag == True):
    temp = original+smaller 
    if original.hex() == temp.hex(): 
        print("Found precision =", smaller)
        flag = False
    smaller = smaller/2

Found precision = 1.7763568394002505e-15


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [19]:
import math

def quadratic_eq(a, b, c):
    x1 = (-b+math.sqrt(b**2-4*a*c))/(2*a)
    x2 = (-b-math.sqrt(b**2-4*a*c))/(2*a)
    return(x1, x2)

def quadratic_eq_mult(a, b, c):
    multiply_x1 = -b-math.sqrt(b**2-4*a*c)
    multiply_x2 = -b+math.sqrt(b**2-4*a*c)
    print("\nFactors to multiply num and denom of x1 and x2 =", multiply_x1, multiply_x2)
    x1 = ((-b+math.sqrt(b**2-4*a*c))*multiply_x1)/((2*a)*multiply_x1)
    x2 = ((-b-math.sqrt(b**2-4*a*c))*multiply_x2)/((2*a)*multiply_x2)
    return(x1, x2)

def quadratic_eq_mult_corr(a, b, c):
    delta = b**2-4*a*c
    if delta > 0: 
        print("\nReal and different roots =") 
        x1 = (-b+math.sqrt(delta))/(2*a)
        x2 = (-b-math.sqrt(delta))/(2*a)
        return(x1, x2)
      
    elif delta == 0: 
        print("\nReal and same roots =")
        x1 = x2 = -b/(2*a)
        return(x1, x2)
      
    # when discriminant is less than 0
    else:
        print("\nComplex Roots") 
        x1 = -b/(2*a), " + i", sqrt_val
        x2 = -b/(2*a), " - i", sqrt_val
        return(x1, x2)

r1 = quadratic_eq(0.001, 1000, 0.001)

r2 = quadratic_eq_mult(0.001, 1000, 0.001)

print("result1 =" + str(r1))
print("\nresult2 =" + str(r2))
print("\nAs we can see x1 remains the same, while x2 changes and is greater than before.")
print("The reason is that float have limited precision. Even by multiplying -b-math.sqrt(b**2-4*a*c)")
print("both at the num and den we can't hold the result of the fraction at 1.")
r3 = quadratic_eq_mult_corr(0.001, 1000, 0.001)
print("result3 =" + str(r3))


Factors to multiply num and denom of x1 and x2 = -1999.999999998 -1.999978849198669e-09
result1 =(-9.999894245993346e-07, -999999.999999)

result2 =(-9.999894245993346e-07, -999999.9999990001)

As we can see x1 remains the same, while x2 changes and is greater than before.
The reason is that float have limited precision. Even by multiplying -b-math.sqrt(b**2-4*a*c)
both at the num and den we can't hold the result of the fraction at 1.

Real and different roots =
result3 =(-9.999894245993346e-07, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [17]:
# Define a number x
x = 1

# function f(x)
def f(x):
    return x*(x-1)

e = 2
delta = pow(10, -e)

y = (f(x+delta)-f(x))/delta
print("For delta =", delta, ":", y)

for e in range(-4, -16, -2):
    delta = pow(10, e)
    y = (f(x+delta)-f(x))/delta
    print("For delta =", delta, ":", y)
    
print("\nBy decreasing delta we can notice that the accuracy of the result increases,")
print("although more iterations can't be calculated by the computer.")

For delta = 0.01 : 1.010000000000001
For delta = 0.0001 : 1.0000999999998899
For delta = 1e-06 : 1.0000009999177333
For delta = 1e-08 : 1.0000000039225287
For delta = 1e-10 : 1.000000082840371
For delta = 1e-12 : 1.0000889005833413
For delta = 1e-14 : 0.9992007221626509

By decreasing delta we can notice that the accuracy of the result increases,
although more iterations can't be calculated by the computer.


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [22]:
import math
import time

N1 = 100
h1 = 2/N1
I1 = 0


for k in range(1, N1+1):                #yk = #value of k-th slice
    yk = math.sqrt(1-(h1*k-1)**2)       #iteration for N=100
    I1 = I1+abs(h1*yk)
print("For N =", k, "I =", I1)



k = 0
N2 = 180000000
h2 = 2/N2
I2 = 0

start = time.time()

for k in range(1, N2+1):
    yk = math.sqrt(1-(h2*k-1)**2)          #iteration for 60s
    I2 = I2+abs(h2*yk)
end = time.time()
print(f"\nRuntime of the program is {end - start}s")
print("For N=", k, "I =", I2)

k = 0
N3 = 3000000
h3 = 2/N3
I3 = 0

start = time.time()

for k in range(1, N3+1):
    yk = math.sqrt(1-(h3*k-1)**2)       #iteration for 1s
    I3 = I3+abs(h3*yk)
end = time.time()

print(f"\nRuntime of the program is {end - start}s")
print("For N=", k, "I =", I3)
print("\nN can be increased up to:", N2)
print("The gain in running 60s instead of 1s is:", I2-I3)


For N = 100 I = 1.5691342555492505

Runtime of the program is 59.56266713142395s
For N= 180000000 I = 1.5707963267946565

Runtime of the program is 1.0193819999694824s
For N= 3000000 I = 1.5707963264747942

N can be increased up to: 180000000
The gain in running 60s instead of 1s is: 3.198623588218652e-10
